In [31]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
path = '../driver/chromedriver.exe'

In [35]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
# # chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-plugins-discovery')
chrome_options.add_argument('--start-maximized')
# chrome_options.add_argument('headless')

browser = webdriver.Chrome(executable_path=path, options=chrome_options)
# url = 'https://www.glassdoor.com/member/home/index.htm'
url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
browser.get(url)
job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))

job_title.clear()
location.clear()

job_title.send_keys('manager')
location.send_keys('Canada')

log_in = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()